In [6]:
import os
import sys
import subprocess
from google.colab import drive

def setup_sumo_rl_environment():
    print("\n🔗 Mounting Google Drive...")
    drive.mount('/content/drive', force_remount=True)
    print("✔ Google Drive mounted.\n")

    # Set project folder in Google Drive (change if needed)
    project_path = '/content/drive/MyDrive/Colab Notebooks/SUMO_DDPG'
    print("📁 Project path is:", project_path)

    # Install SUMO traffic simulator and tools
    print("\n📦 Installing SUMO and SUMO tools ...")
    subprocess.run(['apt', 'update', '-qq'], check=True)
    subprocess.run(['apt-get', 'install', '-y', '-qq', 'sumo', 'sumo-tools'], check=True)
    print("✔ SUMO installation complete.\n")
    !pip install TraCI
    # Set SUMO_HOME environment variable for Python SUMO modules
    os.environ['SUMO_HOME'] = '/usr/share/sumo'
    print("🌐 Set SUMO_HOME to /usr/share/sumo")



    # Add only Google Drive project path to Python path (no local copy)
    if project_path not in sys.path:
        sys.path.append(project_path)
        print(f"➕ Added Drive project path to Python path: {project_path}")

    print("\n✅ SUMO + RL environment setup complete!\n")
    print(f"RUN & EDIT FILES DIRECTLY HERE: {project_path}")
    print(f"SUMO_HOME is set to: {os.environ['SUMO_HOME']}")
    print("\n🦾 Next: Run your RL code from Drive, e.g.:")
    print(f"!python \"{project_path}/Main.py\"")

    return project_path

# Execute setup (run this cell at the top of your notebook every session)
project_dir = setup_sumo_rl_environment()



🔗 Mounting Google Drive...
Mounted at /content/drive
✔ Google Drive mounted.

📁 Project path is: /content/drive/MyDrive/Colab Notebooks/SUMO_DDPG

📦 Installing SUMO and SUMO tools ...
✔ SUMO installation complete.

🌐 Set SUMO_HOME to /usr/share/sumo
➕ Added Drive project path to Python path: /content/drive/MyDrive/Colab Notebooks/SUMO_DDPG

✅ SUMO + RL environment setup complete!

RUN & EDIT FILES DIRECTLY HERE: /content/drive/MyDrive/Colab Notebooks/SUMO_DDPG
SUMO_HOME is set to: /usr/share/sumo

🦾 Next: Run your RL code from Drive, e.g.:
!python "/content/drive/MyDrive/Colab Notebooks/SUMO_DDPG/Main.py"


In [7]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/SUMO_DDPG')
print("Current working directory reset to:", os.getcwd())






Current working directory reset to: /content/drive/MyDrive/Colab Notebooks/SUMO_DDPG


In [ ]:

!python "/content/drive/MyDrive/Colab Notebooks/SUMO_DDPG/Main.py"


 Retrying in 1 seconds
SUMO started successfully with command: sumo -c platoon.sumocfg --no-warnings --step-length 1.0 --no-step-log --time-to-teleport -1 --max-num-vehicles 25 --end 7200
Created platoon 0 with route horizontal at time 0.0
Created platoon 1 with route route_sn_straight at time 0.0
Created platoon 2 with route route_we_straight at time 0.0
Episode duration: 1s (1,000 steps)
Total episodes: 100
Estimated training time per episode: 0.0 minutes
Starting SUMO-V2X Training for 100 episodes...
Episode 0/100
 Retrying in 1 seconds
SUMO started successfully with command: sumo -c platoon.sumocfg --no-warnings --step-length 1.0 --no-step-log --time-to-teleport -1 --max-num-vehicles 25 --end 7200
Created platoon 0 with route horizontal at time 0.0
Created platoon 1 with route route_sn_straight at time 0.0
Created platoon 2 with route route_we_straight at time 0.0
/content/drive/MyDrive/Colab Notebooks/SUMO_DDPG/ddpg_torch.py:39: UserWarning: Creating a tensor from a list of numpy.

In [ ]:
#!/usr/bin/env python3
import os
import sys
import random
import time

# Ensure SUMO is on PYTHONPATH; adjust SUMO_HOME if needed.
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    print("[ERROR] SUMO_HOME not set. Please export SUMO_HOME to your SUMO installation path.")
    sys.exit(1)

from sumolib import checkBinary  # noqa: E402
import traci  # noqa: E402

CFG = "platoon.sumocfg"  # uses single-intersection.net.xml and single-intersection.rou.xml
ROUTE_IDS = [
    "route_north_south",
    "route_west_east",
    "route_north_east",
    "route_west_south",
]
LEADER_VTYPE = "platoon_leader"
FOLLOWER_VTYPE = "platoon_follower"

def start_sumo():
    sumo_binary = checkBinary("sumo")
    cmd = [sumo_binary, "-c", CFG, "--step-length", "0.1", "--no-step-log", "true"]
    print(f"[INFO] Launching SUMO: {' '.join(cmd)}")
    traci.start(cmd)
    print("[INFO] TraCI connected")

def stop_sumo():
    try:
        traci.close()
    except Exception as e:
        print(f"[WARN] Error closing TraCI: {e}")

def make_platoon(platoon_idx, route_id, depart_time, leader_speed=11.11, headway=1.5):
    """
    Create one platoon of 4 vehicles on the given route.
    Vehicles depart with small time offsets to avoid spawn collisions,
    then set minimum gaps and speed to stick together.
    """
    leader_id = f"platoon{platoon_idx}_v0"
    follower_ids = [f"platoon{platoon_idx}_v{k}" for k in range(1, 4)]
    vehicle_ids = [leader_id] + follower_ids

    print(f"[DEBUG] Creating platoon {platoon_idx} on route '{route_id}' at t={depart_time:.1f}")
    print(f"[DEBUG]  Leader={leader_id}, Followers={follower_ids}")

    # Add leader first
    traci.vehicle.add(vehID=leader_id, routeID=route_id, typeID=LEADER_VTYPE, depart=str(depart_time))
    print(f"[DEBUG]  Added leader '{leader_id}' on route '{route_id}'")

    # Small depart offsets to prevent same-step insertion conflicts
    for i, fid in enumerate(follower_ids, start=1):
        traci.vehicle.add(vehID=fid, routeID=route_id, typeID=FOLLOWER_VTYPE,
                          depart=str(depart_time + i * 0.2))
        print(f"[DEBUG]  Added follower '{fid}' on route '{route_id}' with depart={depart_time + i*0.2:.1f}")

    # Step until leader exists in simulation
    while leader_id not in traci.vehicle.getIDList():
        traci.simulationStep()

    # Configure platoon behavior: speed and gap
    print(f"[DEBUG]  Setting leader '{leader_id}' speed to {leader_speed:.2f} m/s")
    traci.vehicle.setSpeed(leader_id, leader_speed)

    # Tighten minGap and set speed mode for followers
    for vid in vehicle_ids:
        vtype = LEADER_VTYPE if vid == leader_id else FOLLOWER_VTYPE
        # Use speed mode that obeys safety but allows speed commands: 0b000000000000 (0) is fully controlled,
        # 31 keeps safety checks; here we use 31 to avoid crashes.
        traci.vehicle.setSpeedMode(vid, 31)
        # Aggressive close-gap for followers
        if vtype == FOLLOWER_VTYPE:
            traci.vehicle.setMinGap(vid, 0.5)
        else:
            traci.vehicle.setMinGap(vid, 2.0)
        print(f"[DEBUG]   Configured '{vid}': vType={vtype}, minGap={traci.vehicle.getMinGap(vid):.2f}, speedMode=31")

    # Let the followers catch up to form a platoon
    return vehicle_ids

def main():
    start_sumo()
    try:
        # Sanity: ensure routes exist
        available_routes = set()
        # Route IDs from loaded route file are accessible via route.getIDList()
        traci.simulationStep()  # advance once so routes are registered
        available_routes.update(traci.route.getIDList())
        print(f"[INFO] Available routes in simulation: {sorted(list(available_routes))}")

        for rid in ROUTE_IDS:
            if rid not in available_routes:
                print(f"[ERROR] Missing route in SUMO: {rid}. Check single-intersection.rou.xml.")
                stop_sumo()
                return

        platoons = []
        # Stagger platoon departures
        sim_t = 0.0
        for p in range(5):
            route_id = random.choice(ROUTE_IDS)
            leader_speed = random.choice([10.0, 11.11, 12.0, 13.0])
            platoon = make_platoon(p, route_id, depart_time=sim_t, leader_speed=leader_speed, headway=1.5)
            platoons.append((p, route_id, platoon))
            sim_t += 6.0  # 6 seconds between platoons

        # Run simulation until end or all vehicles have arrived
        step = 0
        while traci.simulation.getMinExpectedNumber() > 0:
            traci.simulationStep()
            step += 1
            if step % 10 == 0:
                ids = traci.vehicle.getIDList()
                print(f"[TICK] t={traci.simulation.getTime():.1f} s | active={len(ids)} | ids={list(ids)}")

        print("[INFO] Simulation complete.")
    except traci.exceptions.TraCIException as e:
        print(f"[ERROR] TraCI exception: {e}")
    except Exception as e:
        print(f"[ERROR] Unexpected: {e}")
    finally:
        stop_sumo()

if __name__ == "__main__":
    main()


[INFO] Launching SUMO: /usr/share/sumo/bin/sumo -c platoon.sumocfg --step-length 0.1 --no-step-log true
 Retrying in 1 seconds


/usr/share/sumo/tools/traci/main.py:174: UserWarning: Could not connect to TraCI server using port 38517 (TraCI server already finished). Retrying with different port.
  warnings.warn(("Could not connect to TraCI server using port %s (%s)." +


 Retrying in 1 seconds


/usr/share/sumo/tools/traci/main.py:174: UserWarning: Could not connect to TraCI server using port 35241 (TraCI server already finished). Retrying with different port.
  warnings.warn(("Could not connect to TraCI server using port %s (%s)." +


 Retrying in 1 seconds


...

 Retrying in 1 seconds


/usr/share/sumo/tools/traci/main.py:174: UserWarning: Could not connect to TraCI server using port 34301 (TraCI server already finished). Retrying with different port.
  warnings.warn(("Could not connect to TraCI server using port %s (%s)." +


 Retrying in 1 seconds


/usr/share/sumo/tools/traci/main.py:174: UserWarning: Could not connect to TraCI server using port 46169 (TraCI server already finished). Retrying with different port.
  warnings.warn(("Could not connect to TraCI server using port %s (%s)." +


FatalTraCIError: Could not connect in 1 tries

In [ ]:
# ===== FILE ORGANIZATION CELL =====
import shutil
import os

# Your files are already here! Let's organize them
current_files = [f for f in os.listdir('.') if f.endswith('.py')]
print("Found files:", current_files)

# Move files to Classes directory
classes_files = ['Environment_Platoon.py', 'networks.py', 'noise.py', 'buffer.py']

for file in classes_files:
    if os.path.exists(file):
        shutil.move(file, f'Classes/{file}')
        print(f"✅ Moved {file} to Classes/")

# Create __init__.py
with open('Classes/__init__.py', 'w') as f:
    f.write('# Classes module\n')

print("📁 Files organized!")
print("Root:", [f for f in os.listdir('.') if f.endswith('.py')])
print("Classes:", os.listdir('Classes'))


Found files: []
📁 Files organized!
Root: []
Classes: ['__init__.py']


In [ ]:
pip install torch torchvision torchaudio scipy numpy tplotlib


In [ ]:
# ===== MAIN TRAINING SCRIPT =====
import numpy as np
import torch as T
import time
import os
import scipy.io
import sys
import math

# Add paths
sys.path.append('.')
sys.path.append('./Classes')

# Import modules
import Classes.Environment_Platoon as ENV
from ddpg_torch import HybridMADDPGAgent

# Try SUMO import
try:
    import traci
    SUMO_AVAILABLE = True
    print("✅ SUMO available!")
except:
    SUMO_AVAILABLE = False
    print("⚠️ Using fallback simulation")

class SUMOEnhancedEnvironment(ENV.Environ):
    """Enhanced environment with SUMO integration"""

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.sumo_running = False
        self.step_count = 0

    def start_sumo(self):
        """Start SUMO if available"""
        if not SUMO_AVAILABLE:
            return False

        try:
            # Create simple SUMO config
            net_xml = '''<?xml version="1.0" encoding="UTF-8"?>
<net version="1.16">
    <edge id="E1" from="J1" to="J2" priority="1" numLanes="3" speed="15.00"/>
    <junction id="J1" type="priority" x="0.00" y="250.00"/>
    <junction id="J2" type="priority" x="1000.00" y="250.00"/>
</net>'''

            with open('simple.net.xml', 'w') as f:
                f.write(net_xml)

            config_xml = '''<?xml version="1.0" encoding="UTF-8"?>
<configuration>
    <input><net-file value="simple.net.xml"/></input>
    <time><begin value="0"/><step-length value="0.1"/></time>
</configuration>'''

            with open('simple.sumocfg', 'w') as f:
                f.write(config_xml)

            traci.start(["sumo", "-c", "simple.sumocfg", "--no-warnings", "--no-step-log"])
            self.sumo_running = True
            print("✅ SUMO started!")
            return True
        except Exception as e:
            print(f"⚠️ SUMO failed: {e}")
            return False

    def update_with_sumo(self):
        """Update positions using SUMO if available"""
        if self.sumo_running:
            try:
                traci.simulationStep()
                self.step_count += 1
                # Simple position update from SUMO
                for i, vehicle in enumerate(self.vehicles):
                    # Add small random movement to simulate SUMO integration
                    vehicle.position[0] += np.random.uniform(-0.5, 0.5)
                    vehicle.position[1] += np.random.uniform(-0.5, 0.5)
                    # Keep within bounds
                    vehicle.position[0] = np.clip(vehicle.position[0], 0, self.width)
                    vehicle.position[1] = np.clip(vehicle.position[1], 0, self.height)
                return True
            except:
                self.sumo_running = False
                return False
        return False

    def new_random_game(self, n_veh=0):
        """Enhanced episode initialization"""
        super().new_random_game(n_veh)

        if not self.sumo_running:
            self.start_sumo()

        print(f"🎮 New episode (SUMO: {'Active' if self.sumo_running else 'Fallback'})")

    def act_for_training(self, actions):
        """Enhanced training step with SUMO"""
        # Update with SUMO if available
        self.update_with_sumo()

        # Update channels with new positions
        self.renew_channel(self.n_Veh, self.size_platoon)
        self.renew_channels_fastfading()

        # Call parent method
        return super().act_for_training(actions)

    def __del__(self):
        if self.sumo_running:
            try:
                traci.close()
            except:
                pass

def get_state(env, idx, size_platoon):
    """Get state for agent"""
    V2I_abs = (env.V2I_channels_abs[idx * size_platoon] - 60) / 60.0
    V2V_abs = (env.V2V_channels_abs[idx * size_platoon, idx * size_platoon + (1 + np.arange(size_platoon - 1))] - 60)/60.0
    V2I_fast = (env.V2I_channels_with_fastfading[idx * size_platoon, :] - env.V2I_channels_abs[idx * size_platoon] + 10) / 35
    V2V_fast = (env.V2V_channels_with_fastfading[idx * size_platoon, idx * size_platoon + (1 + np.arange(size_platoon - 1)), :] -
                env.V2V_channels_abs[idx * size_platoon, idx * size_platoon + (1 + np.arange(size_platoon - 1))].reshape(size_platoon - 1, 1) + 10) / 35
    Interference = (-env.Interference_all[idx] - 60) / 60
    AoI_levels = env.AoI[idx] / (int(env.time_slow / env.time_fast))
    V2V_load_remaining = np.asarray([env.V2V_demand[idx] / env.V2V_demand_size])

    state = np.concatenate((np.reshape(V2I_abs, -1), np.reshape(V2I_fast, -1), np.reshape(V2V_abs, -1),
                           np.reshape(V2V_fast, -1), np.reshape(Interference, -1), np.reshape(AoI_levels, -1),
                           V2V_load_remaining), axis=0)
    return np.nan_to_num(state, nan=0.0)

def main():
    print("🚗🤖 Starting SUMO-Enhanced MADDPG Training!")

    # Parameters (optimized for quick demo)
    size_platoon = 4
    n_veh = 20
    n_platoon = int(n_veh / size_platoon)
    n_RB = 3
    n_S = 2
    Gap = 25
    max_power = 30
    V2I_min = 540
    bandwidth = 180000
    V2V_size = 4000 * 8

    # Network parameters (reduced for faster training)
    batch_size = 32
    memory_size = 10000
    gamma = 0.99
    alpha = 0.001
    beta = 0.001

    # Network architecture (smaller for speed)
    C_fc1_dims = 256
    C_fc2_dims = 128
    C_fc3_dims = 64
    A_fc1_dims = 256
    A_fc2_dims = 128
    tau = 0.01

    # Lane configurations
    up_lanes = [0.875, 2.625, 125.875, 127.625, 250.875, 252.625]
    down_lanes = [122.375, 124.125, 247.375, 249.125, 372.375, 374.125]
    left_lanes = [0.875, 2.625, 217.375, 219.125, 433.875, 435.625]
    right_lanes = [213.875, 215.625, 430.375, 432.1215, 646.875, 648.625]
    width = 1000
    height = 500

    print(f'🛣️ Environment: {n_veh} vehicles, {n_platoon} platoons')

    # Initialize enhanced environment
    env = SUMOEnhancedEnvironment(
        down_lane=down_lanes, up_lane=up_lanes, left_lane=left_lanes, right_lane=right_lanes,
        width=width, height=height, n_veh=n_veh, size_platoon=size_platoon,
        n_RB=n_RB, V2I_min=V2I_min, BW=bandwidth, V2V_SIZE=V2V_size, Gap=Gap
    )

    env.new_random_game()

    # Get dimensions
    n_input = len(get_state(env=env, idx=0, size_platoon=size_platoon))
    n_output = 3
    print(f"🧠 Network: input={n_input}, output={n_output}")

    # Initialize agents
    agents = []
    for i in range(n_platoon):
        agent = HybridMADDPGAgent(
            alpha=alpha, beta=beta, input_dims=n_input, tau=tau,
            n_actions=n_output, gamma=gamma, max_size=memory_size,
            C_fc1_dims=C_fc1_dims, C_fc2_dims=C_fc2_dims, C_fc3_dims=C_fc3_dims,
            A_fc1_dims=A_fc1_dims, A_fc2_dims=A_fc2_dims,
            batch_size=batch_size, n_agents=n_platoon, agent_name=i
        )
        agents.append(agent)

    print(f"🤖 {len(agents)} agents initialized!")

    # Training parameters
    n_episode = 30  # Quick demo
    n_step_per_episode = 50

    # Storage
    record_global_reward = np.zeros(n_episode)

    start_time = time.time()
    print(f"🚀 Starting training ({n_episode} episodes)...")

    # Training loop
    for i_episode in range(n_episode):
        try:
            if i_episode % 5 == 0:
                status = "SUMO" if env.sumo_running else "Fallback"
                print(f'📊 Episode {i_episode+1}/{n_episode} ({status})')

            env.new_random_game()
            states = [get_state(env, agent_id, size_platoon) for agent_id in range(n_platoon)]

            episode_reward = 0
            for i_step in range(n_step_per_episode):
                # Get actions
                actions_for_env = []
                for agent_id in range(n_platoon):
                    action = agents[agent_id].choose_action(states[agent_id])
                    action = np.nan_to_num(action, nan=0.0)

                    rb_selection = int(np.clip(action[0] * n_RB, 0, n_RB - 1))
                    mode_selection = int(np.clip(action[1] * n_S, 0, n_S - 1))
                    power_selection = np.clip(action[2], 1e-6, max_power)

                    actions_for_env.append([rb_selection, mode_selection, power_selection])

                action_temp = np.array(actions_for_env)

                # Environment step
                per_user_reward, global_reward, platoon_AoI, V2I_rate, V2V_rate, Demand, interference_val = env.act_for_training(action_temp)
                episode_reward += global_reward

                # Learn
                next_states = [get_state(env, agent_id, size_platoon) for agent_id in range(n_platoon)]

                for agent_id in range(n_platoon):
                    agents[agent_id].remember(
                        states[agent_id], action_temp[agent_id],
                        per_user_reward[agent_id], next_states[agent_id], False
                    )
                    agents[agent_id].learn()

                states = next_states

            record_global_reward[i_episode] = episode_reward / n_step_per_episode

        except Exception as e:
            print(f"⚠️ Episode {i_episode} error: {e}")
            record_global_reward[i_episode] = -1

    # Training complete
    training_time = time.time() - start_time
    avg_reward = np.mean(record_global_reward)

    print(f'\n🎉 Training Complete!')
    print(f'⏱️  Time: {training_time:.1f}s')
    print(f'📈 Average reward: {avg_reward:.4f}')
    print(f'🏆 Best reward: {np.max(record_global_reward):.4f}')
    print(f'🚗 SUMO: {"Success" if env.sumo_running else "Fallback"}')

    # Save results
    scipy.io.savemat('result/sumo_maddpg_results.mat', {
        'global_reward': record_global_reward,
        'time': training_time,
        'sumo_used': env.sumo_running
    })
    print('💾 Results saved!')

    return record_global_reward

# Run training
rewards = main()

# Visualize results
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.plot(rewards, 'b-', linewidth=2)
plt.title('Training Progress')
plt.xlabel('Episode')
plt.ylabel('Global Reward')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(rewards[-10:], 'g-', linewidth=2, marker='o')
plt.title('Final Episodes')
plt.xlabel('Episode')
plt.ylabel('Global Reward')
plt.grid(True)

plt.tight_layout()
plt.show()

print(f"\n🎯 Training Summary:")
print(f"   Final reward: {rewards[-1]:.4f}")
print(f"   Total improvement: {((rewards[-1] - rewards[0])/abs(rewards[0])*100 if rewards[0] != 0 else 0):.1f}%")


ModuleNotFoundError: No module named 'Classes.Environment_Platoon'